In [1]:
from contextlib import contextmanager
import sqlite3
import yaml

In [2]:
curr_dir = ! pwd
curr_dir = curr_dir[0]
print(curr_dir)
assert curr_dir.endswith("hyperborea3/scripts/spells")

/home/jeremy/projects/hyperborea3/scripts/spells


In [3]:
load_folder = "lvl5"

In [4]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [5]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the <i>acid arrow</i> strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul><li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li></ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. 

In [6]:
files = ! ls {load_folder}/*.yml
files

['lvl5/Advanced_Hypnotism.yml',
 'lvl5/Advanced_Spectral_Phantasm.yml',
 'lvl5/Air-like_Water.yml',
 'lvl5/Air_Walk.yml',
 'lvl5/Animal_Growth.yml',
 'lvl5/Animate_Carrion_III.yml',
 'lvl5/Anti-Magic_Field.yml',
 'lvl5/Anti-Plant_Shell.yml',
 'lvl5/Atonement.yml',
 'lvl5/Breathe_Fire.yml',
 'lvl5/Breathe_Frost.yml',
 'lvl5/Cause_Lycanthropy.yml',
 'lvl5/Cloudkill.yml',
 'lvl5/Commune.yml',
 'lvl5/Commune_with_Nature.yml',
 'lvl5/Contact_Otherworldly_Being.yml',
 'lvl5/Control_Winds.yml',
 'lvl5/Cure_Critical_Wounds.yml',
 'lvl5/Cure_Madness.yml',
 'lvl5/Death.yml',
 'lvl5/Death_Smoke_Cloud.yml',
 'lvl5/Dismissal.yml',
 'lvl5/Dispel_Evil.yml',
 'lvl5/Extend_Spell_II.yml',
 'lvl5/Fabricate.yml',
 'lvl5/Feeblemind.yml',
 'lvl5/Finger_of_Death.yml',
 'lvl5/Flame_Strike.yml',
 'lvl5/Gelatinize_Bones.yml',
 'lvl5/Hold_Monster.yml',
 'lvl5/Ice_Bridge.yml',
 'lvl5/Incite_Chaos.yml',
 'lvl5/Inoculate.yml',
 'lvl5/Interposing_Hand.yml',
 'lvl5/Magic_Jar.yml',
 'lvl5/Major_Creation.yml',
 'lvl5/M

In [7]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [12]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [53, 346]:
        assert spell["Desc"][-4:] == "</p>"



In [13]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [14]:
for f in files:
    print(f"checking {f}")
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    print("success")
    print()

checking lvl5/Advanced_Hypnotism.yml
success

checking lvl5/Advanced_Spectral_Phantasm.yml
success

checking lvl5/Air-like_Water.yml
success

checking lvl5/Air_Walk.yml
success

checking lvl5/Animal_Growth.yml
success

checking lvl5/Animate_Carrion_III.yml
success

checking lvl5/Anti-Magic_Field.yml
success

checking lvl5/Anti-Plant_Shell.yml
success

checking lvl5/Atonement.yml
success

checking lvl5/Breathe_Fire.yml
success

checking lvl5/Breathe_Frost.yml
success

checking lvl5/Cause_Lycanthropy.yml
success

checking lvl5/Cloudkill.yml
success

checking lvl5/Commune.yml
success

checking lvl5/Commune_with_Nature.yml
success

checking lvl5/Contact_Otherworldly_Being.yml
success

checking lvl5/Control_Winds.yml
success

checking lvl5/Cure_Critical_Wounds.yml
success

checking lvl5/Cure_Madness.yml
success

checking lvl5/Death.yml
success

checking lvl5/Death_Smoke_Cloud.yml
success

checking lvl5/Dismissal.yml
success

checking lvl5/Dispel_Evil.yml
success

checking lvl5/Extend_Spell_

Do it for real

In [15]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [16]:
load_yml_spell(files[-1])

{'id': 439,
 'name': 'Wall of Thorns',
 'pp': 245,
 'Lvl': ['drd 5'],
 'Rng': '90 feet',
 'Dur': '1 turn per CA level',
 'Rev': 0,
 'Desc': '<p>Evokes a giant mass of gnarly, resilient, pliable brush to take form. These thick, tangled vines bristle with dagger-like thorns of three- to five-inch length. The sorcerer may shape the barrier as desired, as large as 1,000 cubic feet per CA level; for instance, a CA 10 sorcerer may create a <i>wall of thorns</i> 50 feet long, 20 feet deep, and 10 feet high. Creatures caught in the spell’s designated area of effect suffer 1d10+10 hp damage. If one attempts to push through the <i>wall of thorns</i> or otherwise comes into abrupt contact with it, similar damage is sustained and repeated for every 10 feet of movement within the mass.</p> <p>To avoid additional damage, one can chop through with a stout blade, such as a sword, axe, or heavy knife. Cutting through 10 feet of this enchanted barrier requires 1 turn. Mundane fire will not harm the <i>w